In [111]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

In [112]:
df=pd.read_csv('F:\Python Prac\sales_data_sample.csv',usecols=['ORDERDATE','QUANTITYORDERED','SALES','PRODUCTLINE','PRODUCTCODE','CITY','STATE','POSTALCODE','COUNTRY'],parse_dates=['ORDERDATE'])

In [113]:
month=df.groupby(['ORDERDATE','PRODUCTLINE','PRODUCTCODE','CITY','STATE','POSTALCODE','COUNTRY']).agg({'SALES':'sum','QUANTITYORDERED':'mean'})

In [114]:
year=df.groupby(['ORDERDATE','PRODUCTLINE','PRODUCTCODE','CITY','STATE','POSTALCODE','COUNTRY']).agg({'SALES':'sum','QUANTITYORDERED':'mean'})

In [115]:
month.reset_index(inplace=True)

In [116]:
year.reset_index(inplace=True)

In [117]:
def monthBeginDate(row):
    return datetime.date(row['ORDERDATE'].year,row['ORDERDATE'].month,1)

In [118]:
def yearBeginDate(row):
    return datetime.date(row['ORDERDATE'].year,1,1)

In [119]:
month['monthBeginDate']=month.apply(monthBeginDate,axis=1)

In [120]:
year['yearBeginDate']=year.apply(yearBeginDate,axis=1)

In [121]:
month_1=month.groupby(['monthBeginDate','PRODUCTLINE','PRODUCTCODE','CITY','STATE','POSTALCODE','COUNTRY']).agg({'SALES':'sum','QUANTITYORDERED':'mean'})

In [122]:
year_1=year.groupby(['yearBeginDate','PRODUCTLINE','PRODUCTCODE','CITY','STATE','POSTALCODE','COUNTRY']).agg({'SALES':'sum','QUANTITYORDERED':'mean'})

In [123]:
month_1.reset_index(inplace=True)

In [124]:
year_1.reset_index(inplace=True)

In [125]:
month_1=month_1.rename(columns={'monthBeginDate':'ORDERDATE'})

In [126]:
year_1=year_1.rename(columns={'yearBeginDate':'ORDERDATE'})

In [127]:
month_2=month_1.groupby(['ORDERDATE','COUNTRY','PRODUCTLINE']).agg({'SALES':'sum','QUANTITYORDERED':'mean'})

In [128]:
year_2=year_1.groupby(['ORDERDATE','COUNTRY','PRODUCTLINE']).agg({'SALES':'sum','QUANTITYORDERED':'mean'})

In [129]:
month_2.reset_index(inplace=True)

In [130]:
year_2.reset_index(inplace=True)

In [131]:
pstDateM=month_2['ORDERDATE'].max()

In [132]:
pstDateY=year_2['ORDERDATE'].max()

In [133]:
preDateM=pstDateM-relativedelta(months=1)

In [134]:
preDateY=pstDateY-relativedelta(years=1)

In [135]:
pstMonth=month_2[month_2['ORDERDATE']==pstDateM]

In [136]:
preMonth=month_2[month_2['ORDERDATE']==preDateM]

In [137]:
pstYear=year_2[year_2['ORDERDATE']==pstDateY]

In [138]:
preYear=year_2[year_2['ORDERDATE']==preDateY]

In [139]:
merged_dataM=pd.merge(preMonth,pstMonth,how='outer',on='COUNTRY')[['COUNTRY','PRODUCTLINE_x','SALES_x','PRODUCTLINE_y','SALES_y']].fillna(0)

In [140]:
merged_dataY=pd.merge(preYear,pstYear,how='outer',on='COUNTRY')[['COUNTRY','PRODUCTLINE_x','SALES_x','PRODUCTLINE_y','SALES_y']].fillna(0)

In [141]:
d1=merged_dataM.groupby(['COUNTRY']).agg({'SALES_x':'sum','SALES_y':'sum'})

In [142]:
d2=merged_dataY.groupby(['COUNTRY']).agg({'SALES_x':'sum','SALES_y':'sum'})

In [143]:
d1['increased']=d1['SALES_y']-d1['SALES_x']

In [144]:
d2['increased']=d2['SALES_y']-d2['SALES_x']

In [160]:
d1

,SALES_x,SALES_y,increased
COUNTRY,,,
Australia,0.00,58349.52,58349.52
Austria,0.00,59475.10,59475.10
Belgium,0.00,8411.95,8411.95
Canada,0.00,33692.97,33692.97
Denmark,26012.87,0.00,-26012.87
France,99494.30,87569.38,-11924.92
Italy,0.00,41509.94,41509.94
Japan,553.95,0.00,-553.95
Singapore,2587.58,0.00,-2587.58


In [163]:
d2

,SALES_x,SALES_y,increased
COUNTRY,,,
Australia,1394380.08,870551.82,-523828.26
Austria,103388.78,273001.04,169612.26
Belgium,188775.06,125200.55,-63574.51
Canada,452771.04,435931.08,-16839.96
Denmark,120431.56,156077.22,35645.66
Finland,589992.55,380555.13,-209437.42
France,3886390.90,1700694.80,-2185696.10
Germany,150418.78,0.00,-150418.78
Ireland,72699.52,0.00,-72699.52


In [158]:
d1.loc[(d1.increased<=0)]

,SALES_x,SALES_y,increased
COUNTRY,,,
Denmark,26012.87,0.00,-26012.87
France,99494.30,87569.38,-11924.92
Japan,553.95,0.00,-553.95
Singapore,2587.58,0.00,-2587.58
UK,40802.81,0.00,-40802.81
USA,567715.72,466969.55,-100746.17
